In [1]:
import pandas as pd
import numpy as np

numRuns = 100

i = np.arange(10)
x = 50 + i
y = 5*i + 100
z = np.arange(35)

X = []
Y = []
Z = []


for i in z:
    X = np.append(X, x)
    Y = np.append(Y, y)
    Z = np.append(Z, np.repeat(i,len(x)))


X = np.tile(x,len(z))
Y = np.tile(y,len(z))
Z = np.repeat(z,len(x))
dpNum = np.arange(len(X))
numDesignPoints = len(dpNum)


r = np.random.randint(0,(2**16)-1,numRuns)
runNum = np.arange(numRuns)

DP = np.repeat(dpNum, numRuns)
X = np.repeat(X,numRuns)
Y = np.repeat(Y,numRuns)
Z = np.repeat(Z,numRuns)

R = np.tile(r,numDesignPoints)
RN = np.tile(runNum, numDesignPoints)


dxpRows = pd.DataFrame({'DP':DP,'Run':RN,'x':X,'y':Y,'z':Z,'randomSeed':R})
    


This is a DXP like dataset.  It has two sweeps, the inner loop is X and Y, the outer loop is Z.  Then each of those sweeps is repeated for a number or runs.

In [2]:
dxpRows

,DP,Run,x,y,z,randomSeed
0,0,0,50,100,0,36059
1,0,1,50,100,0,7202
2,0,2,50,100,0,52578
3,0,3,50,100,0,65485
4,0,4,50,100,0,13468
...,...,...,...,...,...,...
34995,349,95,59,145,34,42120
34996,349,96,59,145,34,24477
34997,349,97,59,145,34,44358
34998,349,98,59,145,34,58497


Next we convert this into a structure that can hold status and run data.  I'm storing status as a string for the example, but it really should be an enumeration

In [9]:
#makes the expStatus

expStatus = dxpRows.copy()
expStatus['Status'] = 'unsubmitted'
#expStatus['ExecutionTime'] = None
#expStatus['ExecutionSimTime'] = None
#expStatus['SimWallTimeRatio'] = None
#expStatus['StartTime'] = None
#expStatus['StopTime'] = None
#expStatus['Directory'] = None

baseExpStatus = expStatus.copy()

In [10]:
#Defines the Cheat GUI

import random
from datetime import datetime, timedelta
from collections import OrderedDict
from math import ceil

def resetExperiment():
    global expStatus
    global baseExpStatus
    expStatus = baseExpStatus.copy()

def updateExperiment(pSubmit, pStart, pFail, pComplete, pUpload):
    global expStatus
    
    #complete a set of uploads
    df = expStatus[expStatus['Status'] == 'uploading']
    ind = random.choices(df.index, k=ceil(len(df) * pUpload/100))
    if len(ind) > 0:
        expStatus.loc[ind,'Status'] = 'uploaded'
    print('%d New Uploaded' % len(ind))
    
    #migrate all the complete to uploading
    df = expStatus[expStatus['Status'] == 'complete']
    expStatus.loc[df.index, 'Status'] = 'uploading'
    print('%d New Uploading' % len(df))
  
    #fail a set of executions
    df = expStatus[expStatus['Status'] == 'executing']
    ind = random.choices(df.index, k=ceil(len(df) * pFail/100))
    if len(ind) > 0:
        expStatus.loc[ind,'Status'] = 'failed'    
    print('%d New Failed' % len(ind))
        
    #complete a set of executions
    df = expStatus[expStatus['Status'] == 'executing']
    ind = random.choices(df.index, k=ceil(len(df) * pComplete/100))
    if len(ind) > 0:
        expStatus.loc[ind,'Status'] = 'complete'
    print('%d New Complete' % len(ind))

    #start queued runs
    df = expStatus[expStatus['Status'] == 'queued']
    ind = random.choices(df.index, k=ceil(len(df) * pStart/100))
    if len(ind) > 0:
        expStatus.loc[ind,'Status'] = 'executing'        
    print('%d New Executing' % len(ind))
    
    #submit a set
    df = expStatus[expStatus['Status'] == 'unsubmitted']
    n = ceil(len(df) * pSubmit/100)
    ind = df.index[:n]
    if len(ind) > 0:
        expStatus.loc[ind,'Status'] = 'queued'  
    print('%d New Queued' % len(ind))

class cheatGUI:
    
    def __init__(self, refreshFcn):
        self.refreshFcn = refreshFcn
    
        ws = []
        ws.append(widgets.Label('Cheat Settings:'))

        self.pSubmit = widgets.FloatSlider(
            value=0,
            min=0,
            max=100.0,
            step=1,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        ws.append(widgets.HBox([widgets.Label('Percent Submitted:'), self.pSubmit]))

        self.pStart = widgets.FloatSlider(
            value=25,
            min=0,
            max=100.0,
            step=1,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        ws.append(widgets.HBox([widgets.Label('Percent Started:'), self.pStart]))

        self.pFail = widgets.FloatSlider(
            value=5,
            min=0,
            max=100.0,
            step=1,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        ws.append(widgets.HBox([widgets.Label('Percent Execution Fail:'), self.pFail]))

        self.pComplete = widgets.FloatSlider(
            value=85.0,
            min=0,
            max=100.0,
            step=1,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        ws.append(widgets.HBox([widgets.Label('Percent Complete Execution:'), self.pComplete]))

        self.pUpload = widgets.FloatSlider(
            value=90.0,
            min=0,
            max=100.0,
            step=1,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        ws.append(widgets.HBox([widgets.Label('Percent Complete Upload:'), self.pUpload]))

        updateBtn = widgets.Button(
            description='Update Progress',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Updates the status',
            icon='' # (FontAwesome names without the `fa-` prefix)
        )

        updateBtn.on_click( self.doUpdate )


        resetBtn = widgets.Button(
            description='Reset Experiment',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='',
            icon='' # (FontAwesome names without the `fa-` prefix)
        )
        
        resetBtn.on_click( self.doReset )

        ws.append(widgets.HBox([updateBtn, resetBtn]))

        self.output = widgets.Output()
        ws.append(self.output)
        
        display(widgets.VBox(ws))
        
    def doUpdate(self, b):
        self.output.clear_output(wait=True)
        with self.output:
            updateExperiment(self.pSubmit.value, 
                             self.pStart.value, 
                             self.pFail.value, 
                             self.pComplete.value, 
                             self.pUpload.value)
            
        self.refreshFcn()
    
    def doReset(self, b):
        self.output.clear_output(wait=True)
        with self.output:
            print('Experiment Reset')
        resetExperiment()
        
        self.refreshFcn()
    



In [11]:
#Defines the Plot Function

from bokeh.plotting import figure, output_notebook, show
output_notebook(resources=None, verbose=False, hide_banner=False, load_timeout=5000, notebook_type='jupyter')

from bokeh.models import FuncTickFormatter
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource

def plotStatusBars(df):
    a = 0.5
    lw = 0.1
    la = 0.5

    p = figure(width=750, height=200, toolbar_location="below", tools = "xpan,xwheel_zoom,xbox_zoom,reset")

    df['index'] = range(len(df))
    source = ColumnDataSource(df)
    uploaded = BooleanFilter([True if s == 'uploaded' else False for s in source.data['Status']])
    uploading = BooleanFilter([True if s == 'uploading' else False for s in source.data['Status']])
    complete = BooleanFilter([True if s == 'complete' else False for s in source.data['Status']])
    executing = BooleanFilter([True if s == 'executing' else False for s in source.data['Status']])
    queued = BooleanFilter([True if s == 'queued' else False for s in source.data['Status']])
    unsubmitted = BooleanFilter([True if s == 'unsubmitted' else False for s in source.data['Status']])
    failed = BooleanFilter([True if s == 'failed' else False for s in source.data['Status']])

    #df = expStatus[expStatus['Status'] == 'uploaded']
    view = CDSView(source=source, filters=[uploaded])

    p.vbar(x='index', bottom=3, top=5, width=1, color='blue', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)
    p.vbar(x='index', bottom=0, top=3, width=1, color='green', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[uploading])
    p.vbar(x='index', bottom=3, top=3.5, width=1, color='lightseagreen', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)
    p.vbar(x='index', bottom=0, top=3, width=1, color='green', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[complete])
    p.vbar(x='index', bottom=0, top=3, width=1, color='green', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[executing])
    p.vbar(x='index', bottom=0, top=1.5, width=1, color='lightgreen', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[queued])
    p.vbar(x='index', bottom=0, top=1, width=1, color='silver', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[unsubmitted])
    p.vbar(x='index', bottom=0, top=0.1, width=1, color='gainsboro', line_width=lw, alpha=a, source=source, view=view)

    view = CDSView(source=source, filters=[failed])
    p.vbar(x='index', bottom=-1, top=0, width=1, color='red', line_width=lw, line_alpha=la, alpha=a, source=source, view=view)



    p.xgrid.grid_line_color = None

    p.x_range.start = 0
    p.x_range.end = len(df)
    p.y_range.start = -1
    p.y_range.end = 5

    p.yaxis.formatter = FuncTickFormatter(code="""
        switch(tick) {
            case -1:
                return 'Failed'
                break;
            case 0:
                return 'Unsubmitted'
                break;
            case 1:
                return 'Queued'
                break;
            case 1.5:
                return 'Executing'
                break;
            case 3:
                return 'Complete'
                break;
            case 3.5:
                return 'Uploading'
                break;
            default:
                return 'Uploaded'
        }
    """)

    p.yaxis.ticker = [-1, 0, 1, 1.5, 3, 3.5, 5]
    p.xaxis.ticker = []

    show(p)


Loading BokehJS ...

In [12]:
#defines a check pagination function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

def dfPageWidget(df):

    from math import ceil
    nPerPage = 5
    nPages = ceil(len(df)/nPerPage)
    

    if len(df) == 0:
        print('No Matches Found!')
        return
    
    print('%d Matches Found' % len(df))    
    
    def pageDf(page=1):
        nDf = len(df)
        startIndex = (page-1)*nPerPage
        if startIndex < nDf:
            stopIndex = startIndex + nPerPage
            if stopIndex >= nDf:
                display(df.iloc[startIndex:])
            else:
                display(df.iloc[startIndex:stopIndex])
        else:
            raise ValueError(page)
        return(page)

   
    pageSlider = widgets.IntSlider(min=1, max=nPages)
        
    output = widgets.interactive_output(pageDf, {'page':pageSlider})
    if nPages > 1:
        display(widgets.VBox([output, pageSlider]))
    else:
        display(output)

In [13]:
#defines the exampleGUI

import ipywidgets as widgets
from IPython.display import display
from collections import OrderedDict
from functools import partial

buttons = {}
labels = ['Unsubmitted', 'Queued', 'Executing', 'Complete', 'Uploading', 'Uploaded', 'Failed' ]

filters = OrderedDict()

class valueNotFilter:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        self.enabled = False
    def doFilter(self, df):
        if self.enabled:
            return df[df[self.column] != self.value]
        else:
            return df
        
holdRefresh = False
expStatusShow = expStatus.copy()

def refresh():
    global holdRefresh
    global expStatusShow
    if holdRefresh:
        return
    
    expStatusShow = expStatus.copy()
    for f in filters.values():
        expStatusShow = f.doFilter(expStatusShow)
        
    orderByColumns = [orderBy.value]
    
    if thenOrderBy.value != orderBy.value:
        orderByColumns.append(thenOrderBy.value)
    
    if 'DP' not in orderByColumns:
        orderByColumns.append('DP')
        
    if 'Run' not in orderByColumns:
        orderByColumns.append('Run') 
               
    if len(expStatusShow) > 1:
        expStatusShow = expStatusShow.sort_values(by=orderByColumns, ascending=ascendingBy.value == 'v' )  #,ignore_index=True  

    output1.clear_output(wait=True)
    with output1:
        plotStatusBars(expStatusShow)
        
    output2.clear_output(wait=True)
    with output2:
        dfPageWidget(expStatusShow)
        
    
        
        
def checkCallback(name, value):
    filters[name].enabled = not value['new']
    if value['new'] != value['old']:
        refresh()
        
        
for label in labels:
    buttons[label] = widgets.Checkbox(
        value=True,
        description=label,
        disabled=False,
        indent=False,
    )
    filters[label] = valueNotFilter('Status', label.lower())
    buttons[label].observe(partial(checkCallback, label), names='value')
    
selectAll = widgets.Button(
    description='Select All',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Selects All Statuses',
    icon='' # (FontAwesome names without the `fa-` prefix)
) 

unselectAll = widgets.Button(
    description='Unselect All',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Selects All Statuses',
    icon='' # (FontAwesome names without the `fa-` prefix)
) 

def selectAllCallback(value, b):
    global holdRefresh
    holdRefresh = True
    for b in buttons.values():
        b.value = value
        
    holdRefresh = False
    refresh()

selectAll.on_click(partial(selectAllCallback, True))
unselectAll.on_click(partial(selectAllCallback, False))

checkBoxes = widgets.HBox([widgets.VBox([buttons['Unsubmitted'], buttons['Queued'], buttons['Executing']]),
                           widgets.VBox([buttons['Complete'], buttons['Uploading'], buttons['Uploaded']]),
                           widgets.VBox([buttons['Failed']])])

statusBox = widgets.VBox([checkBoxes, widgets.HBox([selectAll, unselectAll])])

#---
class selectFilter:
    def __init__(self, column, value='All'):
        self.column = column
        self.value = value
        self.enabled = True
    def doFilter(self, df):
        if self.enabled and self.value != 'All':
            return df[df[self.column] == int(self.value)]
        else:
            return df

def dropdownCallback(name, value):
    filters[name].value = value['new'] 
    if value['new'] != value['old']:
        refresh()        
        
dpSelect = widgets.Dropdown(
    options=['All'] + [str(n) for n in dxpRows['DP'].unique()],
    value='All',
    description='Design Point',
    disabled=False,
)

filters['DP'] = selectFilter('DP')
dpSelect.observe(partial(dropdownCallback, 'DP'), names='value')

runSelect = widgets.Dropdown(
    options=['All'] + [str(n) for n in dxpRows['Run'].unique()],
    value='All',
    description='Runs',
    disabled=False,
)

filters['Run'] = selectFilter('Run')
runSelect.observe(partial(dropdownCallback, 'Run'), names='value')

dpRunBox = widgets.VBox([dpSelect, runSelect])

#--- random

class randomFilter:
    def __init__(self, n, enabled=True):
        self.n = n
        self.enabled = enabled
        self.samples = []
        
    def doFilter(self, df):
        if not self.enabled:
            return df
        
        iInd = np.intersect1d(df.index, self.samples)
        if len(self.samples) == self.n and len(iInd) == self.n:
            #keep old samples
            return df.loc[iInd]
        elif len(df) > 0:
            newDf = df.sample(n=min(len(df), self.n))
            self.samples = newDf.index
            return newDf
        else:
            return df

rndSelect = widgets.RadioButtons(
    options=['All Selected', 'Random Samples'],
    description='',
    disabled=False
)


rndNum = widgets.IntText(
    value=10,
    description='Number:',
    disabled=True
)

rndButton = widgets.Button(
    description='New Samples',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submits selected runs',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

filters['rnd'] = randomFilter(n=10, enabled=False)


def rndSelectObserve(value):
    if value['new'] == 'All Selected':
        rndNum.disabled = True
        rndButton.disabled = True
        filters['rnd'].enabled = False
    else:
        rndNum.disabled = False
        rndButton.disabled = False 
        filters['rnd'].n = rndNum.value
        filters['rnd'].enabled = True
        
    if value['new'] != value['old']:
        refresh()

def rndValueChange(value):
    filters['rnd'].n = value['new']
    if value['new'] != value['old']:
        refresh()    

rndSelect.observe(rndSelectObserve, names='value')
rndNum.observe(rndValueChange, names='value')

def doResample(b):
    filters['rnd'].samples = []
    refresh()

rndButton.on_click(doResample)

rndRunBox = widgets.VBox([rndSelect, widgets.HBox([rndNum, rndButton])])

tbd1 = widgets.Output()
with tbd1:
    print('TBD - by Factor Values')
    
tbd2 = widgets.Output()
with tbd2:
    print('TBD - by Advanced Equation Selection')    

orderBy = widgets.Dropdown(
    options= [str(n) for n in expStatus.columns],
    value='DP',
    description='Order By:',
    disabled=False,
)

thenOrderBy = widgets.Dropdown(
    options= [str(n) for n in expStatus.columns],
    value='Run',
    description='Order By:',
    disabled=False,
)

ascendingBy = widgets.Dropdown(
    options= ['^','v'],
    value='v',
    description='Direction:',
    disabled=False,
)


def orderByCallback(value):
    if value['new'] != value['old']:
        refresh()

orderBy.observe(orderByCallback, names='value')
thenOrderBy.observe(orderByCallback, names='value')     
ascendingBy.observe(orderByCallback, names='value')    

orderBox = widgets.VBox([orderBy, thenOrderBy, ascendingBy])

tab = widgets.Tab(children = [statusBox, dpRunBox, rndRunBox, tbd1, tbd2, orderBox])
tab.set_title(0, 'By Status')
tab.set_title(1, 'By DP/Run')
tab.set_title(2, 'Random Samples')
tab.set_title(3, 'By Factors')
tab.set_title(4, 'Advanced Selection')
tab.set_title(5, 'Order By')

output1 = widgets.Output()
with output1:
    plotStatusBars(expStatus)

output2 = widgets.Output()

with output2:
    dfPageWidget(expStatus)
    
#group actions...
submitBtn = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submits selected runs',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def doSubmit(b):
    global expStatus
    global expStatusShow
    
    #all items unsubmitted become queued
    df = expStatusShow[expStatusShow['Status'] == 'unsubmitted']
    expStatus.loc[df.index,'Status'] = 'queued'
    
    refresh()    

submitBtn.on_click(doSubmit)


cancelBtn = widgets.Button(
    description='Cancel',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cancel Processes for Selected Runs',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def doCancel(b):
    global expStatus
    global expStatusShow
    
    #all items execting become unsubmitted
    df = expStatusShow[expStatusShow['Status'] == 'executing']
    expStatus.loc[df.index,'Status'] = 'unsubmitted'
    
    #all items queued become unsubmitted
    df = expStatusShow[expStatusShow['Status'] == 'queued']
    expStatus.loc[df.index,'Status'] = 'unsubmitted'
    
    #all items uploading become completed (not uploading)
    df = expStatusShow[expStatusShow['Status'] == 'uploading']
    expStatus.loc[df.index,'Status'] = 'complete'
    
    refresh()
    
    
cancelBtn.on_click(doCancel)



deleteBtn = widgets.Button(
    description='Delete',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Delete Results for Selected Runs and convert to unsubmitted',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def doDelete(b):
    global expStatus
    global expStatusShow
    
    #all items become unsubmitted
    expStatus.loc[expStatusShow.index,'Status'] = 'unsubmitted'
    
    refresh()

    
deleteBtn.on_click(doDelete)    
    
uploadBtn = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Uploads Completed Runs',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def doUpload(b):
    global expStatus
    global expStatusShow    
    
    #all items complete become uploading
    df = expStatusShow[expStatusShow['Status'] == 'complete']
    expStatus.loc[df.index,'Status'] = 'uploading'
    
    refresh()
    
    
exportBtn = widgets.Button(
    description='Export...',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Exports Result Summary To CSV File',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

def doExport(b):
    global expStatus
    global expStatusShow
    
    #all items become unsubmitted
    expStatusShow.to_csv('BatchSelection.csv')

    
exportBtn.on_click(doExport)   

uploadBtn.on_click(doUpload)

tx = widgets.Label(value='Group Actions:')
groupActions = widgets.VBox([tx, widgets.HBox([submitBtn, cancelBtn, uploadBtn, deleteBtn, exportBtn]) ])

display(widgets.VBox([tab, output1, output2, groupActions]))

In [14]:
cheatGUI(refresh)
